<img src="https://raw.githubusercontent.com/determined-ai/determined/master/determined-logo.png" align='right' width=150 />

# Building a Pedestrian Detection Model with Determined

<img src="https://www.cis.upenn.edu/~jshi/ped_html/images/PennPed00071_1.png" width=400 />


This notebook will walk through the benefits of building a Deep Learning model with Determined.  We will build an object detection model trained on the [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/) and deploy it to Algorithmia for serving.

In [ ]:
import torch
import torchvision

from determined.experimental import Determined

# Train

## Train a model on the Determined cluster

For our first example, we run a simple single-GPU training job with fixed hyperparameters.

In [ ]:
!det e create const.yaml .

## Determined Model Registry

After training, we'll want to actually use our model in some sort of system.  Determined provides a model registry to version your trained models, making them easy to retrieve for inference.

In [ ]:
experiment_id = < Experiment ID >
MODEL_NAME = "pedestrian_detection_model"

In [ ]:
%%capture
checkpoint = Determined().get_experiment(experiment_id).top_checkpoint()
model = Determined().create_model(MODEL_NAME)
model.register_version(checkpoint.uuid)

# Inference

## Local Inference

Once your model is versioned in the model registry, using that model for inference is straightforward:

In [ ]:
model = Determined().get_model(MODEL_NAME)
trial = model.get_version().load()
inference_model = trial.model

In [ ]:
from predict import predict
predict(inference_model, 'test.jpg', inference="local")

## Serving Endpoint

Now that we can run inference in this notebook, let's set up a serving endpoint on Algorithmia so we can scale this model's serving.

First we'll install Algorithmia and the associated utility packages.

In [ ]:
!pip install Algorithmia
!pip install retry

In [1]:
import Algorithmia
import algorithmia_utils
import urllib.parse
from git import Git, Repo, remote
from retry import retry

### Save your model locally

We'll next save the model from the Determined Model Registry to this current directory.

In [ ]:
torch.save(inference_model, MODEL_NAME)

### Upload your model and sample image

In [2]:
API_KEY = <Your Algorithmia API Key>
USERNAME = <Your Algorithmia username>

client = Algorithmia.client(API_KEY)

In [3]:
DATA_COLLECTION = "pedestrian_detection"
DIRECTORY = client.dir(f"data://{USERNAME}/{DATA_COLLECTION}")
if DIRECTORY.exists() is False:
    DIRECTORY.create()

Once you've created the data collection, you can upload your model and test image to it:

In [ ]:
# Model
MODEL_PATH = f"data://{USERNAME}/{DATA_COLLECTION}/{MODEL_NAME}"
client.file(MODEL_PATH).putFile(MODEL_NAME)

# Test image
TEST_IMG_PATH = f"data://{USERNAME}/{DATA_COLLECTION}/test.jpg"
client.file(TEST_IMG_PATH).putFile("test.jpg")

### Create your serving Algorithm

Algorithmia refers to each endpoint generically as an Algorithm. An Algorithm can be any executable code, in this case it is code that uses our model to generate predictions.

In [4]:
ALGORITHM_NAME = "pedestrian_detection"

In [5]:
algo_utility = algorithmia_utils.AlgorithmiaUtils(API_KEY,
                                                  USERNAME,
                                                  ALGORITHM_NAME,
                                                  local_dir="./")

In [6]:
algo_utility.create_algorithm("pytorch-1.5.x")

Now that the Algorithm is created, we can add the inference code that we want it to execute. Algorithmia supports editing the Algorithm code in your preferred IDE via its Github repo. You an either create a repo within your own Github account (if you link your Github account to Algorithmia), or you can use the repo created for you automatically by Algorithmia. For this example, we'll do the latter.

In [ ]:
algo_utility.clone_algorithm_repo()

Your algorithm repository comes with an initial script that is executed when you post your inference requests. We can edit and write that out to the cloned repo using the `%%writefile` macro. 

Make sure to edit the `model_path` variable in the script below, to point to the uploaded model path in your Algorithmia data source. Since we're creating the script with the %%writefile macro in this notebook, we won't be able to make use of our existing `DATA_COLLECTION` and `MODEL_PATH` variables to dynamically create this script, but will have to copy-paste their values into the cell below.

In [ ]:
%%writefile $algo_utility.algo_script_path
import Algorithmia
import torch
from .predict import predict

#Defining our Algorithmia client in global scope
client = Algorithmia.client()

def load_model(model_path):
    
    model_file = client.file(model_path).getFile().name
    with open(model_file, 'rb') as model_file:
        model = torch.load(model_file, map_location=torch.device('cuda'))
        return model

#Loading our model object in the global scope, outside the apply() function as we don't want do this operation every time our algorithm receives a request and be most efficient as possible
model_path = "data://.my/pedestrian_detection/pedestrian_detection_model"
model = load_model(model_path)


# API calls will begin at the apply() method, with the request body passed as 'input'
# For more details, see algorithmia.com/developers/algorithm-development/languages
def apply(input):
    
    img_path = input["img_path"]
    img = client.file(img_path).getFile().name
    
    with open(img, 'rb') as img:
        predictions = predict(model, img, inference="algorithmia")
    
    return predictions

Similarly, we can create a dependency file to ensure the Algorithm builds the correct runtime environment.

Since we created the algorithm on Algorithmia's Pytorch 1.15 GPU environment, we don't need to include the `torch` dependency in our requirements file. Our algorithm's base environment will be created with that installed and we will be building on top of it.

In [ ]:
%%writefile $algo_utility.dependency_file_path
algorithmia>=1.0.0,<2.0
torchvision==0.6.1
pillow
numpy
matplotlib

For running the actual predictions, we can continue to use the `predict.py` and `data.py` modules that we used earlier. We'll now just copy it directly into the repo so that it can be uploaded to the Algorithm.

In [ ]:
!cp predict.py ./{ALGORITHM_NAME}/src

We're now ready to push the updated repo to Algorithmia.

This operation will trigger the algorithm build at Algorithmia's git server. So you should wait until you see the "Build succesful" message in the output of the next cell, before you continue with sending your inference requests.

In [ ]:
algo_utility.push_algo_script_with_dependencies(filenames=[
    f"{ALGORITHM_NAME}.py",
    "predict.py",
])

With the Algorithm created and inference code pushed, we can run inference on the test image we uploaded earlier.

In [ ]:
algo_result = algo_utility.call_latest_algo_version({
    "img_path": TEST_IMG_PATH
})

In [ ]:
print(algo_result.metadata)

In [ ]:
print(algo_result.result)